In [ ]:
import graphlab

In [ ]:
mec = graphlab.SFrame('data/sppmecdata.csv')

In [1]:
mec['buysig'] = mec['diffmec'] < 0
mec.head()

NameError: name 'mec' is not defined

In [ ]:
graphlab.canvas.set_target('ipynb')

In [ ]:
trdata,tedata = mec.random_split(.8,seed=0)

In [ ]:
sppfeatures = ['loadforecast','windforecast','natgasprice','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
mec[sppfeatures].show()

In [ ]:
mec.show(view='BoxWhisker Plot', x='hour', y='diffmec')

In [ ]:
spp_regression_model = graphlab.linear_regression.create(trdata,target='diffmec',features=sppfeatures,validation_set=tedata)

In [ ]:
print spp_regression_model.evaluate(tedata)

In [ ]:
tint = {'windforecast':[7500],
        'loadforecast':[28777],
        'natgasprice':[3.20],
        '0':[0],
        '1':[0],
        '2':[0],
        '3':[0],
        '4':[0],
        '5':[0],
        '6':[0],
        '7':[0],
        '8':[1],
        '9':[0],
        '10':[0],
        '11':[0],
        '12':[0],
        '13':[0],
        '14':[0],
        '15':[0],
        '16':[0],
        '17':[0],
        '18':[0],
        '19':[0],
        '20':[0],
        '21':[0],
        '22':[0],
        '23':[0]}

print spp_regression_model.predict(graphlab.SFrame(tint))

In [ ]:


spp_classifier_model = graphlab.logistic_classifier.create(trdata,
                                                     target='buysig',
                                                     features=sppfeatures,
                                                     validation_set=tedata)

In [ ]:

spp_nn_model = graphlab.neuralnet_classifier.create(trdata,
                                                     target='buysig',
                                                     features=sppfeatures,
                                                     validation_set=tedata)

In [ ]:
spp_classifier_model.evaluate(tedata, metric='roc_curve')

In [ ]:
spp_classifier_model.show(view='Evaluation')

In [ ]:
mec['predicted_buysig'] = spp_classifier_model.predict(mec, output_type='probability')
reg = spp_regression_model.predict(graphlab.SFrame(tint))
cla = spp_classifier_model.predict(graphlab.SFrame(tint))
#nn = spp_nn_model.predict(graphlab.SFrame(tint))
print 'Regression model predicts diffmec of ' + str(reg)
print 'Classifier model predicts a DA buy? (0:Sell , 1:Buy) : ' + str(cla)
#print 'NN model predicts a DA buy? (0:Sell , 1:Buy) : ' + str(nn)

In [2]:
def create_hour(w,l,n,hour):
    return {'windforecast':[w],
        'loadforecast':[l],
        'natgasprice':[n],
        str(hour):[1]}
    

In [3]:
#3/31/2017 pred
wf = [5130,5295,5260,5140,5046,5000,4924,4814,4797,4763,4765,5080,5543,5952,6294,6528,6767,7132,7586,7993,8335,8658,8919,8970]
lf = [24510,23501,22994,22768,22780,23261,24618,26973,28441,28467,28337,28266,28090,27834,27725,27585,27231,27284,27124,26951,26965,27327,27327,26701,25464]
ng = 3.20
for h in range(0,24):
    reg = spp_regression_model.predict(graphlab.SFrame(create_hour(wf[h],lf[h],ng,h)))
    cla = spp_classifier_model.predict(graphlab.SFrame(create_hour(wf[h],lf[h],ng,h)))
    
    if reg[0] < -2 and cla[0] == 1:
        print 'Hour ' + str(h)
        print '---------------------'
        print 'STRONG BUY'
        print 'Regression model predicts diffmec of ' + str(reg)
        print 'Classifier model predicts a DA buy? (0:Sell , 1:Buy) : ' + str(cla)
        print ''
    elif reg[0] > 2 and cla[0] == 0:
        print 'Hour ' + str(h)
        print '---------------------'
        print 'STRONG SELL'
        print 'Regression model predicts diffmec of ' + str(reg)
        print 'Classifier model predicts a DA buy? (0:Sell , 1:Buy) : ' + str(cla)
        print ''
    #else:
        #print 'Hour ' + str(h)
        #print '---------------------'
        #print 'MIXED'
        #print ''

NameError: name 'spp_regression_model' is not defined